In [80]:
import requests
import shutil
import os
import cv2
import subprocess
import datetime
import re
import glob
import time
import pandas as pd
import numpy as np
from datetime import datetime
import re

videoDirectory = "D:\DystoniaCoalition\RawVideos\\"
profilingOutputDirectory = "D:\DystoniaCoalition\processed\\"


start = datetime.now() #for time logging

df = pd.DataFrame(columns=['NAME', 'SIZE_MB','HEIGHT','WIDTH','FPS','FRAME_COUNT', 'duration', 'PID'])
videos =  os.listdir(videoDirectory)
for idx, filex in enumerate(videos):
    # print(media_info.tracks[0].duration)
    row = [None] * df.columns.size
    vid = cv2.VideoCapture( os.path.join(videoDirectory, filex))
    row[0] = filex
    row[1] = os.path.getsize(os.path.join(videoDirectory, filex)) *0.000001
    row[2] = vid.get(cv2.CAP_PROP_FRAME_HEIGHT) 
    row[3]  = vid.get(cv2.CAP_PROP_FRAME_WIDTH)  
    row[4]  = vid.get(cv2.CAP_PROP_FPS)  
    row[5]  = vid.get(cv2.CAP_PROP_FRAME_COUNT)  
    if cv2.CAP_PROP_FRAME_COUNT and vid.get(cv2.CAP_PROP_FPS):
        row[6]  = vid.get(cv2.CAP_PROP_FRAME_COUNT) / vid.get(cv2.CAP_PROP_FPS)
    underSplit = filex.lower().split('_')
    if (len(underSplit)>1):
        row[7] =int(underSplit[0].split('dys')[1])
    else:
        row[7] =int(filex.lower().split(' ')[0].split('dys')[1])
    df.loc[len(df)] = row
# df.to_csv(profilingOutputDirectory+"videoProfile.csv", index=False)

In [114]:
Patient_Details = pd.read_excel("D:\DystoniaCoalition\dataset\P1.LV.Koirala.2021.3.30-2_edited.xlsx", sheet_name='Patient Details')
Patient_Details['PID'] = Patient_Details['Participant ID'].apply(lambda x: int(x.lower().split('dys')[1]))
print(Patient_Details.columns)
Patient_Detail =  Patient_Details[[  'PID', 
    # 'Participant ID', 
    'Visit Date', 'Gender ', 'Age in years',
#        'Age at onset of Dystonia', 'Disease Duration at Visit',
#        'Race1 (AI: American Indian, AN: Alaska Native, NH: Native Hawaiian, PI: Pacific Islander)',
#        'Race2 (AI: American Indian, AN: Alaska Native, NH: Native Hawaiian, PI: Pacific Islander)',
#        'Additional Race Information', 'Ethnicity ', 
       'Handedness', 'Affected Status', 
    #    'Best Clinical Diagnosis',
       'Type of Dystonia ', 'Subtype ',
    #    'Describe Other Dystonia Syndrome(optional)',
    #    'Has the participant evolved to non-isolated dystonia?',
    #    'If evolved, describe', 'Country of Birth',
      ]]

Index(['Participant ID', 'Visit Date', 'Gender ', 'Age in years',
       'Age at onset of Dystonia', 'Disease Duration at Visit',
       'Race1 (AI: American Indian, AN: Alaska Native, NH: Native Hawaiian, PI: Pacific Islander)',
       'Race2 (AI: American Indian, AN: Alaska Native, NH: Native Hawaiian, PI: Pacific Islander)',
       'Additional Race Information', 'Ethnicity ', 'Ethnicity, if 'Other'',
       'Handedness', 'Affected Status', 'Best Clinical Diagnosis',
       'Type of Dystonia ', 'Subtype ',
       'Describe Other Dystonia Syndrome(optional)',
       'Has the participant evolved to non-isolated dystonia?',
       'If evolved, describe', 'Country of Birth', 'PID'],
      dtype='object')


In [116]:
merged = pd.merge(df, Patient_Detail, on='PID', how='left')
print(merged.columns)
merged  =merged.drop(['Affected Status' ], 1)
# merged['PID'] = merged['PID'].apply(lambda x: '<a href="/SingleGraph/'+str(x)+'">'+str(x)+'</a>')
merged['FRAME_COUNT'] = merged['FRAME_COUNT'].apply(lambda x: int(x))
merged['HEIGHT'] = merged['HEIGHT'].apply(lambda x: int(x))
merged['WIDTH'] = merged['WIDTH'].apply(lambda x: int(x))
merged['Gender '] = merged['Gender '].apply(lambda x: str(x).title())
merged['Age in years'] = merged['Age in years'].apply(lambda x: 0 if str(x)=='nan' else int(x))
merged['Size'] = merged.apply(lambda x: str(x["HEIGHT"])+ str("x")+ str(x['WIDTH']), axis=1)
merged['Type of Dystonia '] = merged['Type of Dystonia '].apply(lambda x: str(x).title())
merged['Subtype '] = merged['Subtype '].apply(lambda x: ' '.join(map(str,(filter(len, re.split(';|-|,|:',str(x).title().replace("Nan", "").replace("Select", "").replace(" ", "") )))))
)
merged

Index(['NAME', 'SIZE_MB', 'HEIGHT', 'WIDTH', 'FPS', 'FRAME_COUNT', 'duration',
       'PID', 'Visit Date', 'Gender ', 'Age in years', 'Handedness',
       'Affected Status', 'Type of Dystonia ', 'Subtype '],
      dtype='object')


C:\Users\susan\AppData\Local\Temp\ipykernel_23656\171770813.py:3: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only



,NAME,SIZE_MB,HEIGHT,WIDTH,FPS,FRAME_COUNT,duration,PID,Visit Date,Gender,Age in years,Handedness,Type of Dystonia,Subtype,Size
0,!DYS3316_20180514.mp4,95.813632,360,640,29.969295,37758,1259.8895,3316,2018-05-14,Male,58,Right,Focal Dystonia,CranialDystonia BlepharospasmOnly,360x640
1,DYS 1101_20150706.mp4,0.001245,0,0,0.000000,0,None,1101,2015-07-06,Female,40,Right,Focal Dystonia,CranialDystonia BlepharospasmOnly,0x0
2,DYS1001_20170222.mp4,100.635770,480,640,29.970001,13238,441.708367,1001,2017-02-22,Female,84,Right,Focal Dystonia,CervicalDystonia,480x640
3,DYS1002_20121219.mp4,184.060641,480,640,29.970009,24234,808.608367,1002,2012-12-19,Female,54,Right,Focal Dystonia,CervicalDystonia,480x640
4,DYS1003_20170322.mp4,142.749892,480,640,29.970016,18802,627.360367,1003,2017-03-22,Male,53,Right,Focal Dystonia,CervicalDystonia,480x640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,DYS988_20121203.mp4,342.331210,1080,1920,24.990829,16350,654.24,988,2012-12-03,Female,57,Right,Focal Dystonia,CervicalDystonia,1080x1920
431,DYS989_20121203.mp4,148.304974,480,640,19.945695,12983,650.917393,989,2012-12-03,Male,63,Right,Focal Dystonia,LimbDystonia UpperExtremityDystonia,480x640
432,DYS993_20130102.mp4,124.341145,480,640,29.970035,16384,546.679367,993,2013-01-02,Male,22,Right,Focal Dystonia,CervicalDystonia,480x640
433,DYS994_20121213.mp4,190.803841,360,640,29.969981,25110,837.838367,994,2012-12-13,Female,58,Right,Focal Dystonia,CervicalDystonia,360x640


In [117]:
# merged.to_csv(profilingOutputDirectory+"videoProfile.csv", index=False)

mergedExport  =merged.drop(['NAME',], 1)

C:\Users\susan\AppData\Local\Temp\ipykernel_23656\4169188719.py:3: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only



In [151]:
from pandas_profiling import ProfileReport

profile = ProfileReport(mergedExport, title="Dystonia Dataset Profiling Report", explorative=True, correlations=None,    missing_diagrams=None,     interactions=None,)
# profile.to_file(output_file=profilingOutputDirectory+"videoProfiling.html")
# videowithScoreDataset

#Lets add Score to the videoProfile


In [120]:
GDRS = pd.read_excel("D:\DystoniaCoalition\dataset\P1.LV.Koirala.2021.3.30-2_edited.xlsx", sheet_name='GDRS')
GDRS.columns
GDRS.drop(GDRS.columns[[1,2,3,4]],axis=1,inplace=True)
GDRS.rename(columns={ GDRS.columns[0]: "PID" }, inplace = True)
GDRS = GDRS.loc[:, ~GDRS.columns.str.startswith('Unnamed')].iloc[1:].reset_index(drop=True)

GDRS['PID'] = GDRS['PID'].apply(lambda x: int(re.findall("\d+", x)[0]))

GDRS.dtypes

PID                                   int64
Eyes and Upper Face                  object
Lower Face                           object
Jaw and Tongue                       object
Larynx                               object
 Neck                                object
 Shoulder and Proximal Arm - Left    object
Shoulder and Proximal Arm - Right    object
Elbow/Distal Arm/Hand - Left         object
 Elbow/Distal Arm/Hand - Right       object
Pelvis and Upper Leg - Left          object
Pelvis and Upper Leg - Right         object
Distal Leg and Foot - Left           object
Distal Leg and Foot - Right          object
Trunk                                object
dtype: object

In [148]:
# pd.merge(merged, GDRS, on='PID', how='right')
videowithScoreDataset = pd.merge(merged, GDRS, on='PID', how='right')
videowithScoreDataset.dropna(subset=[' Neck'], inplace=True) # remove if score is not judged.  . for ex. neck
# videowithScoreDataset.columns
videowithScoreDataset = videowithScoreDataset[videowithScoreDataset["FPS"] > 0]  #remove is FPS is NAN
videowithScoreDataset = videowithScoreDataset.astype({"PID": int,})
videowithScoreDataset = videowithScoreDataset.astype({"PID": str,})

videowithScoreDataset.to_csv(profilingOutputDirectory+"videoProfile.csv", index=False)


In [132]:
# # from pandasgui import show
# # show(videowithScoreDataset)
# len(videowithScoreDataset)
# len(df.dropna())

